# Homework 4 - Variational Quantum Simulation

In this exercise, you will run a Variational Quantum Eignesolver simualtion of the Hydrogen (H$_2$) molecule. The goal will be to see how well the algorithm performs depending on the choice of parameterized wavefunction, or *ansatz*.

For this work, we will need to install two additional libraries, PySCF and Qiskit Nature, which provide utilities for constructing models of atoms and molecules for quantum chemistry simulaitons. Run the next cell to install these libraries.

## Part 1 - Coherence Times and Fidelity

In [14]:
from sympy.solvers import solve
from sympy import Symbol
import numpy as np

In [ ]:
# Initial variable declarations
m1_T1 = 800 * pow(10, -3)
m1_T2 = 250 * pow(10, -3)
m1_GT = 100 * pow(10, -6)
m1_EPLG = 0.003

m2_T1 = 150 * pow(10, -6)
m2_T2 = 60 * pow(10, -6)
m2_GT = 80 * pow(10, -9)
m2_EPLG = 0.017

#### Question 1

In [ ]:
x = Symbol('x')
e = np.e
m1_T1_pow = (-1 * m1_GT) / m1_T1
m1_T1_fidelity = float(solve(pow(e, m1_T1_pow * x) - 0.5, x)[0])
m1_T2_pow = (-1 * m1_GT) / m1_T2
m1_T2_fidelity = float(solve(pow(e, m1_T2_pow * x) - 0.5, x)[0])

m2_T1_pow = (-1 * m2_GT) / m2_T1
m2_T1_fidelity = float(solve(pow(e, m2_T1_pow * x) - 0.5, x)[0])
m2_T2_pow = (-1 * m2_GT) / m2_T2
m2_T2_fidelity = float(solve(pow(e, m2_T2_pow * x) - 0.5, x)[0])
m2_T2_fidelity

print(f"Machine 1:\nT1: {m1_T1_fidelity}\tT2: {m1_T2_fidelity}")
print(f"Machine 2:\nT1: {m2_T1_fidelity}\tT2: {m2_T2_fidelity}")

#### Question 2

In [ ]:
m1_2Q_fidelity = float(solve(m1_EPLG * x - 0.5, x)[0])
m2_2Q_fidelity = float(solve(m2_EPLG * x - 0.5, x)[0])
print(f"Machine 1: {m1_2Q_fidelity}")
print(f"Machine 2: {m2_2Q_fidelity}")

#### Question 3

Supposing that cost is no problem, machine 1 should be the obvious choice, since machine 1 has a higher amount of gate fidelity in both single qubit and 2 qubit gates. 

#### Question 4

Even if $T_{1}$ and $T_{2}$ are infinite, those are just coherence times, it doesn't mean the system is error free, since applying gates and measurements would still introduce noise to the system eventually. 

## Part 2 - Programming Exercise

In [ ]:
# Run this cell to install Qiskit Nature, which is needed for the hydrogen VQE simulation.
!pip install qiskit_nature

In [ ]:
# OPTIONAL - If you are on Mac or Linux, you can install PySCF to generate the molecular Hamiltonians
!pip install pyscf

In [1]:
import datetime as dt
import qiskit as qs
import qiskit_aer as aer
from  qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, Session, IBMRuntimeError
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver, ElectronicStructureDriver

### Create a hardware backend and a noise model simulator
The VQE algorithm can take up to a hour to run on the actual quantum machine. If you're having trouble getting through the algorithm, or if the machine has a long waiting time, you may use the noise model simulator by modifying the cell below.

Set `use_simulator` to `True` or `False` depending on which backend you want to use, then run the next 3 cells.

In [17]:
use_simulator = False

In [18]:
# DO NOT MODIFY - Set up the hardware backend

service = QiskitRuntimeService()
hardware = service.backend('ibm_rensselaer')

pm = generate_preset_pass_manager(backend=hardware, optimization_level=3)

simulator = aer.AerSimulator.from_backend(hardware)

if use_simulator:
    backend = simulator

else:
    backend = hardware

In [19]:
# DO NOT MODIFY - This cell defines utility functions for timing and transpilation
def get_timestamp():
    """
    Returns the current UTC timestamp in ISO-8601 format.

    Returns
    -------
    str
    """
    return dt.datetime.now(dt.UTC).isoformat()


def transpile_and_apply(ansatz, hamiltonian, backend, optimization_level=3, passmanager=None):
    """
    Transpile the ansatz circuit and apply the Hamiltonian observable to the backend layout.
    """

    if not passmanager:
        passmanager = generate_preset_pass_manager(target=backend.target,
                                                   optimization_level=optimization_level)
    
    isa_ansatz = passmanager.run(ansatz)
    isa_hamiltonian = hamiltonian.apply_layout(isa_ansatz.layout)

    return isa_ansatz, isa_hamiltonian

## Simulation Setup
We start by defining the hydrogen molecule system according to a particular physics model defined by `PySCF`. The function defined in the next cell creates a hydrogen molecule with a bond length between the atoms, which is defined as an optional default parameter.

Once the atoms are defined, `PySCF` creates a `problem` instance from which we can extract the system Hamiltonian, which is the matrix operator that describes the physics of the molecule. This is what we will use to define the quantum measurement operator in a later step.

**Important!** If you are on Windows, PySCF is not supported outside of Windows Subsystem for Linux. If you cannot run the next cell, run the one immediately after, which has the Hamiltonian hard-coded.

In [20]:
def create_h2_molecule(bond_length=0.735):
    """
    Creates a representation of the Hamiltonian representing the physics of the
    hydrogen molecule. The bond length (in Angstroms) is provided as a
    parameter.
    """

    # Fix the first hydrogen atom at the origin.
    atom1 = f"H 0 0 {-bond_length/2}"

    # Set the Z-coordinate of the second atom using the bond length we provide
    atom2 = f"H 0 0 {bond_length/2}"

    # Join the strings in the right format to pass to the driver.
    atom = "; ".join([atom1, atom2])
    
    # Using the PySCF driver, we build the Hamiltonian that describes the
    # physics of the hydrogen molecule.
    driver = PySCFDriver(
        atom=atom,
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    # After the Hamiltonian is constructed, we extract the Hamiltonian as a
    # FermionicOp
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    return problem, hamiltonian

In [21]:
hamiltonian = FermionicOp({'+_0 -_0': -1.25633907300325,
                           '+_1 -_1': -0.47189600728114184,
                           '+_2 -_2': -1.25633907300325,
                           '+_3 -_3': -0.47189600728114184,
                           '+_0 +_0 -_0 -_0': 0.33785507740175813,
                           '+_0 +_1 -_1 -_0': 0.3322908651276482,
                           '+_0 +_2 -_2 -_0': 0.33785507740175813,
                           '+_0 +_3 -_3 -_0': 0.3322908651276482,
                           '+_0 +_0 -_1 -_1': 0.09046559989211572,
                           '+_0 +_1 -_0 -_1': 0.09046559989211572,
                           '+_0 +_2 -_3 -_1': 0.09046559989211572,
                           '+_0 +_3 -_2 -_1': 0.09046559989211572,
                           '+_1 +_0 -_1 -_0': 0.09046559989211572,
                           '+_1 +_1 -_0 -_0': 0.09046559989211572,
                           '+_1 +_2 -_3 -_0': 0.09046559989211572,
                           '+_1 +_3 -_2 -_0': 0.09046559989211572,
                           '+_1 +_0 -_0 -_1': 0.3322908651276482,
                           '+_1 +_1 -_1 -_1': 0.34928686136600884, 
                           '+_1 +_2 -_2 -_1': 0.3322908651276482, 
                           '+_1 +_3 -_3 -_1': 0.34928686136600884, 
                           '+_2 +_0 -_0 -_2': 0.33785507740175813, 
                           '+_2 +_1 -_1 -_2': 0.3322908651276482, 
                           '+_2 +_2 -_2 -_2': 0.33785507740175813, 
                           '+_2 +_3 -_3 -_2': 0.3322908651276482, 
                           '+_2 +_0 -_1 -_3': 0.09046559989211572, 
                           '+_2 +_1 -_0 -_3': 0.09046559989211572, 
                           '+_2 +_2 -_3 -_3': 0.09046559989211572, 
                           '+_2 +_3 -_2 -_3': 0.09046559989211572, 
                           '+_3 +_0 -_1 -_2': 0.09046559989211572, 
                           '+_3 +_1 -_0 -_2': 0.09046559989211572, 
                           '+_3 +_2 -_3 -_2': 0.09046559989211572, 
                           '+_3 +_3 -_2 -_2': 0.09046559989211572, 
                           '+_3 +_0 -_0 -_3': 0.3322908651276482, 
                           '+_3 +_1 -_1 -_3': 0.34928686136600884, 
                           '+_3 +_2 -_2 -_3': 0.3322908651276482, 
                           '+_3 +_3 -_3 -_3': 0.34928686136600884}, 
                          num_spin_orbitals=4, )

### Mapping the problem to qubits
The function defined in the next cell takes the Hamiltonian model from the `PySCF` physics driver and maps it to a Pauli gate representation using the *Jordan-Wigner Transform*. This method converts any arbitrary Hamiltonian matrix into a combination of **X**, **Y**, and **Z** operations with scalar coefficients. This is what lets us run the problem on a quantum computer.

In [22]:
def map_to_qubits(hamiltonian):
    # Create a SparsePauliOp using the Jordan-Wigner transform 
    mapper = JordanWignerMapper()
    mapped_hamiltonian = mapper.map(hamiltonian)

    return mapped_hamiltonian, mapper

### Defining an Ansatz
The ansatz of a problem is the assumed form of the quantum state that the Hamiltonian is operating on. It isn't necessarily a physical wave function, just an assumed one with parameters that we can vary.

In the next cell, we define two ansätze. The first, Unitary Coupled Cluster Singles and Doubles (UCCSD), represents a parameterized wavefunction as a unitary transformation of a reference state using an expansion of single and double excitation operators. Intuitively, this ansatz captures electron correlations by allowing electrons to be promoted from occupied to unoccupied orbitals, and in the quantum simulation, each qubit represents an orbital rather than a particle.

The second ansatz, TwoLocal, represents the wavefunction as a parameterized quantum circuit composed of alternating layers of single-qubit rotations and entangling two-qubit gates. Ths is meant to be more optimal for hardware efficiency, allowing customization of gate types and connectivity to match the capabilities of the quantum device while optimizing over adjustable parameters. In contract to UCCSD, it makes no assumptions of the underlying physics, and is entirely based on the layout of the hardware.

In [23]:
def create_ansatze(mapped_hamiltonian, mapper):

    num_spatial_orbitals = 2
    num_particles = (1, 1)
    
    uccsd_ansatz = UCCSD(num_spatial_orbitals,
                         num_particles,
                         mapper,
                         initial_state=HartreeFock(
                             num_spatial_orbitals,
                             num_particles,
                             mapper,),)

    twolocal_ansatz = TwoLocal(mapped_hamiltonian.num_qubits,        
                           rotation_blocks=["rx", "ry"],
                           entanglement_blocks=["cz"],
                           entanglement="linear",
                           reps=2,
                           initial_state=None,)

    return uccsd_ansatz, twolocal_ansatz

### Energy Function
This is the core module that constructs and executes the quantum circuit, which evaluates Schrodinger's equation, $E\psi = \textbf{H}\psi$ for the given set of parameters (in $\psi$) and fixed Hamiltonian. The end result is an expectation value that represents the ground state energy of the Hamiltonian, which in this exercise, represents the ground state energy of the Hydrogen atom.

In [24]:
def energy_function(params, ansatz, hamiltonian, estimator, session, metadata_dict):
    """
    This cost function returns the potential energy of the bond between two
    Hydrogen atoms using a qiskit estimator.

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    # Create the PUB for this parameter set
    pub = (ansatz, [hamiltonian], [params])

    # Submit the job to the session    
    job = estimator.run(pubs=[pub])

    # Grab the job ID so we can retrieve it later
    job_id = job.job_id()
    metadata_dict['job_ids'].append(job_id)

    # Wait on the result
    result = job.result()

    # Get the expectation value and std. deviation
    energy = result[0].data.evs[0]
    err = result[0].data.stds[0]


    # Keep track of the answers and statistics in the run
    metadata_dict["iters"] += 1
    metadata_dict["params"].append(params)
    metadata_dict["energy"].append((energy, err))

    if metadata_dict["iters"] >= metadata_dict["maxiter"]:
        session.close()
    
    print(f"Iters. done: {metadata_dict['iters']} [Energy: {energy} +/- {err}]")

    return energy

### VQE Algorithm
The cells below define the full VQE algorithm for both ansatze, which use the Scipy `minimize` function to run the energy function in a loop until it converges to a minimum value. The minimum value corresponds to the algorithm's estimate of the ground state energy.

In [25]:
def run_vqe_twolocal(backend, isa_twolocal, isa_hamiltonian_twolocal):

    num_params_twolocal = isa_twolocal.num_parameters
    x0_twolocal = np.zeros(num_params_twolocal)
    
    twolocal_dict = {
        'session_id': None,
        'iters': 0,
        'backend': backend.name,
        'maxiter': 200,
        'params': [],
        'energy': [],
        'job_ids': [],
        'job_info':[],
        'start_time': None,
        'end_time': None,
        'completed': False
    }
    
    with Session(backend=backend, max_time=3600) as session:
    
        # Create the estimator and assign the shot value
        estimator = Estimator(mode=session)
        estimator.options.default_precision = 1e-1
        estimator.options.resilience_level = 1
        estimator.options.default_shots = 8192
    
        try:
            # Execute the minimization
            res = minimize(
                energy_function,
                x0_twolocal,
                args=(isa_twolocal, 
                      isa_hamiltonian_twolocal, 
                      estimator, 
                      session, 
                      twolocal_dict),
                method='COBYLA',
                tol=1e-1,
            )
    
        except IBMRuntimeError:
            print("Session closed after exceeding time limit. Stopping iteration.")
    
    return twolocal_dict


In [26]:
def run_vqe_uccsd(backend, isa_uccsd, isa_hamiltonian_uccsd):

    num_params_uccsd = isa_uccsd.num_parameters
    x0_uccsd = np.zeros(num_params_uccsd)
    
    uccsd_dict = {
        'session_id': None,
        'iters': 0,
        'backend': backend.name,
        'maxiter': 200,
        'params': [],
        'energy': [],
        'job_ids': [],
        'job_info':[],
        'start_time': None,
        'end_time': None,
        'completed': False
    }
    
    with Session(backend=backend, max_time=3600) as session:
    
        # Create the estimator and assign the shot value
        estimator = Estimator(mode=session)
        estimator.options.default_precision = 1e-1
        estimator.options.resilience_level = 1
        estimator.options.default_shots = 8192
    
        try:
            # Execute the minimization
            res = minimize(
                energy_function,
                x0_uccsd,
                args=(isa_uccsd, 
                      isa_hamiltonian_uccsd, 
                      estimator, 
                      session, 
                      uccsd_dict),
                method='COBYLA',
                tol=1e-1,
            )
    
        except IBMRuntimeError:
            print("Session closed after exceeding time limit. Stopping iteration.")
    
    return uccsd_dict

## Setup the Problem
Now that we've defined all the functions, the next cell combines all of that into a problem we can run.

In [27]:
# Create the molecular model and Hamiltonian
mapped_hamiltonian, mapper = map_to_qubits(hamiltonian)

# Create the ansatze
uccsd_ansatz, twolocal_ansatz = create_ansatze(mapped_hamiltonian, mapper)

# Create the quantum circuits for each ansatz combination
isa_uccsd, isa_hamiltonian_uccsd = transpile_and_apply(uccsd_ansatz,
                                                       mapped_hamiltonian,
                                                       backend=backend,
                                                       optimization_level=3,
                                                       passmanager=pm)

isa_twolocal, isa_hamiltonian_twolocal = transpile_and_apply(twolocal_ansatz,
                                                             mapped_hamiltonian,
                                                             backend=backend,
                                                             optimization_level=3,
                                                             passmanager=pm)

## Run the TwoLocal solution
The next cell will execute the VQE algorithm using the TwoLocal ansatz. On quantum hardware, **this may take up to an hour to run**. On a simulator, it will take 5-10 minutes.

In [ ]:
twolocal_results = run_vqe_twolocal(simulator, isa_twolocal, isa_hamiltonian_twolocal)

## Run the UCCSD Solution
The next cell will solve the VQE problem for the UCCSD ansatz. On quantum hardware, **this may take up to an hour to run**. On a simulator, it will take 5-10 minutes.

In [28]:
uccsd_results = run_vqe_uccsd(backend, isa_uccsd, isa_hamiltonian_uccsd)

Iters. done: 1 [Energy: -1.0048572789374273 +/- 0.005940575880150195]
Iters. done: 2 [Energy: -0.8393681018814573 +/- 0.006562722860334721]
Iters. done: 3 [Energy: -0.9739765815729073 +/- 0.006155396209919594]
Iters. done: 4 [Energy: -0.912902829218902 +/- 0.006738050372986608]
Iters. done: 5 [Energy: -0.7687937133691711 +/- 0.006468528260922361]
Iters. done: 6 [Energy: -0.892347467231694 +/- 0.006020461586515172]
Iters. done: 7 [Energy: -1.006604681513333 +/- 0.006598543533941115]
Iters. done: 8 [Energy: -1.0098530438345201 +/- 0.006371953936526306]
Iters. done: 9 [Energy: -1.0156951080275032 +/- 0.006223687900090464]
Iters. done: 10 [Energy: -0.9957923166075975 +/- 0.0064636562906733985]
Iters. done: 11 [Energy: -1.0031756744657694 +/- 0.006165519864689069]
Iters. done: 12 [Energy: -1.0066626596254662 +/- 0.006878544245920304]
Iters. done: 13 [Energy: -1.0115459178406712 +/- 0.00633236674654296]
Iters. done: 14 [Energy: -0.9999062306828669 +/- 0.006334282323083411]


In [ ]:
uccsd_jobs = ['cwkybn6543p00086sw70',
              'cwkyc4r40e0000899p90',
              'cwkyces0r6b0008q2qvg',
              'cwkyctb31we00088cz90',
              'cwkyd4w543p00086swbg',
              'cwkydfn543p00086swcg',
              'cwkydx731we00088czc0',
              'cwkye6840e0000899peg',
              'cwkyegj0r6b0008q2r20',
              'cwkyev331we00088czfg',
              'cwkyf540r6b0008q2r3g',
              'cwkyfex0r6b0008q2r4g',
              'cwkyfsqmptp000861sh0',
              'cwkyg3g9r49g0089cesg']

### Compare to the Reference Value
We can use `numpy.linalg.eigh` to find the ground state of the Hamiltonian directly, since hydrogen is small enough to be solved classically. We use this as the reference value to compare our quantum results to.

In [ ]:
ham = mapped_hamiltonian.to_matrix()
reference_energy = np.linalg.eigh(ham)[0][0]

uccsd_energy, uccsd_err = uccsd_results['energy'][-1]
twolocal_energy, twolocal_err = twolocal_results['energy'][-1]

print(f'Reference Energy: {reference_energy} eV')
print(f'UCCSD Energy: {uccsd_energy} eV')
print(f'TwoLocal Energy: {twolocal_energy} eV')

### Plot the Result
The cell below plots the results of the two quantum calculations alongside the reference energy for comparison.

In [ ]:
xvals = ["Reference", "TwoLocal", "UCCSD"]
yvals = [reference_energy, twolocal_energy, uccsd_energy]
yerr = [0, twolocal_err, uccsd_err]

plt.bar(xvals, yvals)
plt.errorbar(xvals, yvals, yerr, fmt="o", color="r")

#### Question 1
Run the cells in the Jupyter notebook to get the estimated ground state energy for H2 using
the UCCSD and TwoLocal ansatze. Report your answers along with the reference answer.

Reference Energy: -1.8572750302023795 eV

UCCSD Energy: -1.6619710008160649 eV

TwoLocal Energy: -1.781458967248546 eV

#### Question 2
How do your answers for the two quantum calculations compare to the reference answer?

They are both less than the reference value.

#### Question 3
Does one ansatz give an answer closer to the reference value over the other? Which one? Why
might this be?

The twolocal answer gives a closer value to the reference, and the reason for that might be due to the greater number of iterations done. That would result in a closer answer given that any Monte Carlo simulation would get closer to the true answer the more iterations it has.

#### Question 4
Define the terms ansatz and Hamiltonian. What does each one represent in the simulation?

Ansatz is German for "an educated guess". In science, it is an initial assumption that's used to solve a problem. A Hamiltonian represents the total energy of a system. In the simulation, the ansatz represents the initial guess of what the energy is would end up looking like, and the Hamiltonian is the total energy state of the hydrogen atom. 

#### Question 5
You are leading a quantum research team performing ground state calculations of a complex molecule. You just ran this $H_{2}$ simulation as a benchmark to inform what ansatz you should pick for the bigger calculation. What ansatz would you choose if your team is prioritizing
cost-effectiveness? Why?

The UCCSD ansatz would be used for cost-effectivness, since it managed to roughly approximate the energy levels of $H_{2}$ with only 12 iterations, as opposed to twolocal's 143 iterations. 

## Part 3 - Free Response: Project Proposal

Team members: Jeremy Chen, Nathaniel Depue, Nate Mirin, Sameer Premji

Our group is planning on creating a grant proposal for making a quantum sensor. The plan is to, at the very least, by the time project 2 is due, to a have a fully outline proposal for a making a quantum sensor. We wish to approach professor Tom Haley for a grant, and moving forward from there.

As for contribution, I will be handling the information analysis part of the quantum sensor. Since the information gathered from the sensor would be messy and needs to be parsed, an algorithm for processes like denoising and squeezing. Though I don't know how that will be accomplished right now, my idea is to have the input be a sequence of signal strengths to be plotted, and that data will be crunch and processed afterwards with some sort of Python library. 